In [1]:
import pymc3 as pm
import theano
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data.csv')
train = data[:2985]
test = data[2986:]

In [3]:
ranks=4
team_number = 20
player_names = set(train.name)
opp_defense_rank_no = train.opp_defense_rank.max() 
opp_attack_rank_no = train.opp_attack_rank.max() 
team_cluster_rank_no = train.team_cluster_rank.max() 
opp_cluster_rank_no = train.opp_cluster_rank.max() 
num_positions = 4
N = len(train)


with pm.Model() as model:
    nu = pm.Exponential('nu minus one', 1/29.,shape=2) + 1 
    err = pm.Uniform('std dev based on rank', 0, 100, shape=ranks)
    err_b = pm.Uniform('std dev based on rank b', 0, 100, shape=ranks)

    # Theano shared variables to change at test time
    was_home_True = theano.shared(np.asarray(train['was_home_True'].values, dtype = bool))
    was_home_False = theano.shared(np.asarray(train['was_home_False'].values, dtype = bool))
    opp_cluster_rank = theano.shared(np.asarray((train['opp_cluster_rank']).values, dtype = int))
    opp_defense_rank = theano.shared(np.asarray((train['opp_defense_rank']).values, dtype = int))
    opp_attack_rank = theano.shared(np.asarray((train['opp_attack_rank']).values, dtype = int))
    initval = theano.shared(np.asarray((train['initval']).values, dtype = int))
    player_home = theano.shared(np.asarray(train['was_home'].values, dtype = int))
    player_avg = theano.shared(np.asarray((train['game_avg_7']).values, dtype = float))
    player_opp = theano.shared(np.asarray((train['opponent_team']).values, dtype = int))
    player_team = theano.shared(np.asarray((train['team']).values, dtype = int))
    player_rank = theano.shared(np.asarray((train['rank']-1).values, dtype = int))
    position_FWD = theano.shared(np.asarray((train['position_FWD']).values.astype(int), 
                                            dtype = int))
    position_MID = theano.shared(np.asarray((train['position_MID']).values.astype(int), 
                                            dtype = int))
    position_GK = theano.shared(np.asarray((train['position_GK']).values.astype(int), 
                                           dtype = int))
    position_DEF = theano.shared(np.asarray((train['position_DEF']).values.astype(int), 
                                            dtype = int))
    pos_id = theano.shared(np.asarray((train['pos_id']).values, dtype = int))

    # Defensive ability of the opposing team vs. each position, partially pooled
    opp_def = pm.Normal('opp team prior',0, sd=100**2, shape=num_positions)
    opp_fwd = pm.Normal('defensive differential fwd', opp_def[0], sd=100**2, shape=team_number)
    opp_mid = pm.Normal('defensive differential mid', opp_def[1], sd=100**2, shape=team_number)
    opp_gk = pm.Normal('defensive differential gk', opp_def[2], sd=100**2, shape=team_number)
    opp_defe = pm.Normal('defensive differential defe', opp_def[3], sd=100**2, shape=team_number)
    
    # Partially pooled ability of the player's rank partially pooled based on position
    home_adv = pm.Normal('home additivie prior', 0, 100**2,shape = num_positions)     
    away_adv = pm.Normal('away additivie prior', 0, 100**2,shape = num_positions)     
    pos_home_fwd = pm.Normal('home differential fwd',home_adv[0],10**2, shape = ranks)
    pos_home_gk = pm.Normal('home differential gk',home_adv[1],10**2, shape = ranks)
    pos_home_defe = pm.Normal('home differential defe',home_adv[2],10**2, shape = ranks)
    pos_home_mid = pm.Normal('home differential mid',home_adv[3],10**2, shape = ranks)
    pos_away_fwd = pm.Normal('away differential fwd',away_adv[0],10**2, shape = ranks)
    pos_away_gk = pm.Normal('away differential gk',away_adv[1],10**2, shape = ranks)
    pos_away_mid = pm.Normal('away differential mid',away_adv[2],10**2, shape = ranks)
    pos_away_defe = pm.Normal('away differential defe',away_adv[3],10**2, shape = ranks)

In [4]:
with model:
    
    team_strength = pm.Normal('team_strength',0, sd=100**2, shape=2) #home and away
    team_strength_home = pm.Normal('team_strength_home',team_strength[0],
                                   sd=100, shape=num_positions)
    team_strength_away = pm.Normal('team_strength_away',team_strength[1], 
                                   sd=100, shape=num_positions)

    team_strength_home_FWD = pm.Normal('team_strength_home_FWD',0, 
                                       sd=100, shape=team_cluster_rank_no)
    team_strength_home_MID = pm.Normal('team_strength_home_MID',0,
                                       sd=100, shape=team_cluster_rank_no)
    team_strength_home_DEF = pm.Normal('team_strength_home_DEF',0, 
                                       sd=100, shape=team_cluster_rank_no)
    team_strength_home_GK = pm.Normal('team_strength_home_GK',0, 
                                      sd=100, shape=team_cluster_rank_no)
    
    team_strength_away_FWD = pm.Normal('team_strength_away_FWD',0, 
                                       sd=100, shape=team_cluster_rank_no)
    team_strength_away_MID = pm.Normal('team_strength_away_MID',0, 
                                       sd=100, shape=team_cluster_rank_no)
    team_strength_away_DEF = pm.Normal('team_strength_away_DEF',0, 
                                       sd=100, shape=team_cluster_rank_no)
    team_strength_away_GK = pm.Normal('team_strength_away_GK',0,
                                      sd=100, shape=team_cluster_rank_no)
    
    
    team_strength_effects = (
        was_home_True*position_FWD*team_strength_home_FWD[opp_cluster_rank-1] +
        was_home_True*position_MID*team_strength_home_MID[opp_cluster_rank-1] +
        was_home_True*position_DEF*team_strength_home_DEF[opp_cluster_rank-1] +
        was_home_True*position_GK*team_strength_home_GK[opp_cluster_rank-1] +

        was_home_False*position_FWD*team_strength_away_FWD[opp_cluster_rank-1] +
        was_home_False*position_MID*team_strength_away_MID[opp_cluster_rank-1] +
        was_home_False*position_DEF*team_strength_away_DEF[opp_cluster_rank-1] +
        was_home_False*position_GK*team_strength_away_GK[opp_cluster_rank-1])
    
    opp_strength = pm.Normal('opp_strength',0, sd=100**2, shape=3)
    opp_strength_team = pm.Normal('opp_strength_team',0, sd=100**2, shape=num_positions)
    opp_strength_defense_team = pm.Normal('opp_strength_defense_team',0, sd=100**2, shape=num_positions)
    opp_strength_attack_team = pm.Normal('opp_strength_attack_team',0, 
                                         sd=100**2, shape=num_positions)
    
    opp_strength_team_FWD = pm.Normal('opp_strength_team_FWD',0, 
                                      sd=100**2, shape=opp_cluster_rank_no)
    opp_strength_team_MID = pm.Normal('opp_strength_team_MID',0, 
                                      sd=100**2, shape=opp_cluster_rank_no)
    opp_strength_team_DEF = pm.Normal('opp_strength_team_DEF',0, 
                                      sd=100**2, shape=opp_cluster_rank_no)
    opp_strength_team_GK = pm.Normal('opp_strength_team_GK',0, 
                                     sd=100**2, shape=opp_cluster_rank_no)
    
    opp_strength_team_effects = (
        position_FWD*opp_strength_team_FWD[opp_cluster_rank-1] +
        position_MID*opp_strength_team_MID[opp_cluster_rank-1] +
        position_DEF*opp_strength_team_DEF[opp_cluster_rank-1] +
        position_GK*opp_strength_team_GK[opp_cluster_rank-1])
                
    opp_strength_defense_team_FWD = pm.Normal('opp_strength_defense_team_FWD',0, 
                                              sd=100**2, shape=opp_defense_rank_no)
    opp_strength_defense_team_MID = pm.Normal('opp_strength_defense_team_MID',0, 
                                              sd=100**2, shape=opp_defense_rank_no)
    opp_strength_defense_team_DEF = pm.Normal('opp_strength_defense_team_DEF',0, 
                                              sd=100**2, shape=opp_defense_rank_no)
    opp_strength_defense_team_GK = pm.Normal('opp_strength_defense_team_GK',0, 
                                             sd=100**2, shape=opp_defense_rank_no)

    opp_strength_defense_team_effects = (
        position_FWD*opp_strength_defense_team_FWD[opp_cluster_rank-1] +
        position_MID*opp_strength_defense_team_MID[opp_cluster_rank-1] +
        position_DEF*opp_strength_defense_team_DEF[opp_cluster_rank-1] +
        position_GK*opp_strength_defense_team_GK[opp_cluster_rank-1])
                
    opp_strength_attack_team_FWD = pm.Normal('opp_strength_attack_team_FWD',0, 
                                             sd=100**2, shape=opp_attack_rank_no)
    opp_strength_attack_team_MID = pm.Normal('opp_strength_attack_team_MID',0, 
                                             sd=100**2, shape=opp_attack_rank_no)
    opp_strength_attack_team_DEF = pm.Normal('opp_strength_attack_team_DEF',0, 
                                             sd=100**2, shape=opp_attack_rank_no)
    opp_strength_attack_team_GK = pm.Normal('opp_strength_attack_team_GK',0, 
                                            sd=100**2, shape=opp_attack_rank_no)

    opp_strength_attack_team_effects = (
        position_FWD*opp_strength_attack_team_FWD[opp_cluster_rank-1] +
        position_MID*opp_strength_attack_team_MID[opp_cluster_rank-1] +
        position_DEF*opp_strength_attack_team_DEF[opp_cluster_rank-1] +
        position_GK*opp_strength_attack_team_GK[opp_cluster_rank-1])


In [5]:
with model:
    # First likelihood where the player's difference from average is explained by defensive abililty
    def_effect = position_FWD*opp_fwd[player_opp-1]+ 
    position_MID*opp_mid[player_opp-1]+ position_GK*opp_gk[player_opp-1]+ 
    position_DEF*opp_defe[player_opp-1]
    
    like1 = pm.StudentT('Diff From Avg', mu=def_effect, 
                        sd=err_b[player_rank],nu=nu[1], observed = train['diff_from_avg'])
    
    initval_coeff = pm.Normal('initval_coeff',0,100, shape = N)
    
    # Second likelihood where the score is predicted by defensive power plus other smaller factors
    mu = player_avg + def_effect
    
    mu += position_GK*pos_home_gk[player_rank]*(player_home) +
    position_MID*pos_home_mid[player_rank]*(player_home) 
    
    mu += position_FWD*pos_home_fwd[player_rank]*(player_home) +
    position_DEF*pos_home_defe[player_rank]*(player_home) 
    
    mu += position_GK*pos_away_gk[player_rank]*(1-player_home) +
    position_MID*pos_away_mid[player_rank]*(1-player_home) 
    
    mu += position_FWD*pos_away_fwd[player_rank]*(1-player_home) +
    position_DEF*pos_away_defe[player_rank]*(1-player_home)  
    
    mu += (team_strength_effects + opp_strength_team_effects +
           opp_strength_attack_team_effects + 
           opp_strength_defense_team_effects + initval*initval_coeff)
    
    like2 = pm.StudentT('Score', mu=mu, sd=err[player_rank], 
                        nu=nu[0], observed=train['total_points'])

    # Training!
    trace=pm.sample(10000, pm.Metropolis())
    
    

<ipython-input-5-70ce0aedf037>:20: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace=pm.sample(10000, pm.Metropolis())
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [initval_coeff]
>Metropolis: [opp_strength_attack_team_GK]
>Metropolis: [opp_strength_attack_team_DEF]
>Metropolis: [opp_strength_attack_team_MID]
>Metropolis: [opp_strength_attack_team_FWD]
>Metropolis: [opp_strength_defense_team_GK]
>Metropolis: [opp_strength_defense_team_DEF]
>Metropolis: [opp_strength_defense_team_MID]
>Metropolis: [opp_strength_defense_team_FWD]
>Metropolis: [opp_strength_team_GK]
>Metropolis: [opp_strength_team_DEF]
>Metropolis: [opp_strength_team_MID]
>Metropolis: [opp_strength_team_FWD]
>Metropolis: [opp_strength_attack_team]
>Metropolis: [opp_strength_defense_team]
>Metropolis: [opp_strength_team]
>

Sampling 4 chains for 1_000 tune and 10_000 draw iterations (4_000 + 40_000 draws total) took 1942 seconds.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


In [9]:
with model:
    was_home_True = theano.shared(np.asarray(test['was_home_True'].values, dtype = bool))
    was_home_False = theano.shared(np.asarray(test['was_home_False'].values, dtype = bool))
    opp_cluster_rank = theano.shared(np.asarray((test['opp_cluster_rank']).values, dtype = int))
    opp_defense_rank = theano.shared(np.asarray((test['opp_defense_rank']).values, dtype = int))
    opp_attack_rank = theano.shared(np.asarray((test['opp_attack_rank']).values, dtype = int))
    initval = theano.shared(np.asarray((test['initval']).values, dtype = int))
    player_home = theano.shared(np.asarray(test['was_home'].values, dtype = int))
    player_avg = theano.shared(np.asarray((test['game_avg_7']).values, dtype = float))
    player_opp = theano.shared(np.asarray((test['opponent_team']).values, dtype = int))
    player_team = theano.shared(np.asarray((test['team']).values, dtype = int))
    player_rank = theano.shared(np.asarray((test['rank']-1).values, dtype = int))
    position_FWD = theano.shared(np.asarray((test['position_FWD']).values.astype(int), 
                                            dtype = int))
    position_MID = theano.shared(np.asarray((test['position_MID']).values.astype(int), 
                                            dtype = int))
    position_GK = theano.shared(np.asarray((test['position_GK']).values.astype(int), 
                                           dtype = int))
    position_DEF = theano.shared(np.asarray((test['position_DEF']).values.astype(int), 
                                            dtype = int))
    pos_id = theano.shared(np.asarray((test['pos_id']).values, 
                                      dtype = int))


    ppc=pm.sample_posterior_predictive(trace, samples=1000)


/Users/meenaljhajharia/anaconda3/lib/python3.8/site-packages/pymc3/sampling.py:1689: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  warnings.warn(


In [10]:
from sklearn.metrics import mean_absolute_error

print('Projection Mean Absolute Error:', 
      mean_absolute_error(test.loc[:,'total_points'].values, ppc['Score'].mean(axis=0)))

print('7 Day Average Mean Absolute Error:', 
      mean_absolute_error(test.loc[:,'total_points'].values, test.loc[:,'game_avg_7'].values))

Projection Mean Absolute Error: 2.219863021260242
7 Day Average Mean Absolute Error: 1.9040440296721703


In [12]:
test['pred_points'] = ppc['Score'][0].tolist()
pts=test.groupby(['name']).sum()
pts.sort_values(by =['pred_points'], inplace = True,ascending=False)
pred = set(pts[:11].index)
pts=test.groupby(['name']).sum()
pts.sort_values(by =['total_points'], inplace = True,ascending=False)
truth = set(pts[:11].index)
len(pred.intersection(truth))

8

In [19]:
pred

{'Emiliano Martínez',
 'Harry Kane',
 'Heung-Min Son',
 'Illan Meslier',
 'Jamie Vardy',
 'Jordan Pickford',
 'Lucas Digne',
 'Matheus Pereira',
 'Mohamed Salah',
 'Patrick Bamford',
 'Sam Johnstone'}

In [20]:
truth

{'Emiliano Martínez',
 'Harry Kane',
 'Illan Meslier',
 'Lucas Digne',
 'Matheus Pereira',
 'Mohamed Salah',
 'Nicolas Pépé',
 'Patrick Bamford',
 'Sam Johnstone',
 'Stuart Dallas',
 'Trent Alexander-Arnold'}

In [21]:
test['pred_points'] = ppc['Score'][0].tolist()
pts=test.groupby(['name']).sum()
pts.sort_values(by =['pred_points'], inplace = True,ascending=False)
pred = set(pts[:100].index)
pts=test.groupby(['name']).sum()
pts.sort_values(by =['total_points'], inplace = True,ascending=False)
truth = set(pts[:100].index)
len(pred.intersection(truth))

78

In [25]:
# pm.save_trace(trace=trace)
# pm.load_trace('.pymc_1.trace')